# Лабораторная работа №3

## "Приближенное вычисление интегралов"

У меня был 1 вариант заданий, следовательно использовались следующие данные:
1. Определенный интеграл
$$ \int_{0}^{1}\frac{ln(1 + x)}{1 + x^2}dx $$
2. Составные квадратурные формулы трапеций и Симпсона
3. Количество узлов равно $$ k = 5 $$
4. Точное значение интеграла равно $$ I = \frac{\pi}{8}ln2 $$

### Задание 1

Вычислить интеграл $ \int_{a}^{b}f(x)dx $ с точностью $ \epsilon = 10^{-7} $ используя составные квадратурные формулы (КФ), указанные в варианте задания, и правило Рунге оценки погрешности. Сравнить полученные приближенные значения интеграла с точным значением $ I $.\
\
В содержание отчета должна быть включена следующая информация:
- Применяемые составные квадратурные формулы.
- Правило Рунге оценки погрешности.
- Результаты вычислительного эксперимента, оформленные в виде таблицы.
- Выводы.
- Листинг программы с комментариями.

#### Импорт библиотек

In [60]:
import numpy as np
import pandas as pd
from scipy.special import roots_legendre

#### Подынтегральная функция

In [63]:
def f(x):
    return np.log(1 + x) / (1 + x**2)

#### Функция квадратурной формулы трапеций

In [66]:
def trapezoidal_rule(a, b, n):
    h = (b - a) / n
    integral = 0.0
    for i in range(1, n):
        x_i = a + i * h
        integral += f(x_i)
    integral += (f(a) + f(b)) / 2
    integral *= h
    return integral

#### Функция квадратурной формулы Симпсона

In [69]:
def simpsons_rule(a, b, n):
    h = (b - a) / n
    integral = f(a) + f(b)
    for i in range(1, n):
        x_i = a + i * h
        if i % 2 == 0:
            integral += 2 * f(x_i)
        else:
            integral += 4 * f(x_i)
    integral *= h / 3
    return integral

#### Функция правила Рунге оценки погрешности

In [72]:
def runge_rule(I1, I2, m):
    return np.abs(I2 - I1) / (2**m - 1)

#### Функция заполнения таблицы с КФ трапеций

In [90]:
def composite_trapezoidal_rule(a, b, epsilon, I, df):
    k = 1
    I1 = trapezoidal_rule(a, b, k)
    while True:
        h = (b - a) / k
        I2 = trapezoidal_rule(a, b, 2 * k)
        error = runge_rule(I1, I2, 2)
        abs_error = np.abs(I - I2)
        df.loc[len(df.index)] = ["Трапеции", 2 * k, h, I2, error, abs_error]
        if error <= epsilon:
            return I2
        I1 = I2
        k *= 2

#### Функция заполнения таблицы с КФ Симпсона

In [93]:
def composite_simpsons_rule(a, b, epsilon, I, df):
    k = 1
    I1 = simpsons_rule(a, b, k)
    while True:
        h = (b - a) / k
        I2 = simpsons_rule(a, b, 2 * k)
        error = runge_rule(I1, I2, 4)
        abs_error = np.abs(I - I2)
        df.loc[len(df.index)] = ["Симпсона", 2 * k, h, I2, error, abs_error]
        if error < epsilon:
            return I2
        I1 = I2
        k *= 2

#### Вызов функций

In [96]:
# Точное значение интеграла
I = (np.pi * np.log(2)) / 8

# Значения a и b
a = 0
b = 1

# Значение эпсилон
eps = 1e-7

# Создаём таблицу
df = pd.DataFrame(columns=["Квадратурная формула",
                           "Число разбиений",
                           "Шаг",
                           "Приближенное значение интеграла",
                           "Оценка погрешности",
                           "Абсолютная погрешность"])

# Вычисление интеграла с помощью квадратурной формулы трапеции
approx_trapezoidal_integral = composite_trapezoidal_rule(a, b, eps, I, df)

# Вычисление интеграла с помощью квадратурной формулы Симпсона
approx_simpsons_integral = composite_simpsons_rule(a, b, eps, I, df)


In [98]:
df

,Квадратурная формула,Число разбиений,Шаг,Приближенное значение интеграла,Оценка погрешности,Абсолютная погрешность
0,Трапеции,2,1.000000,0.248829,2.518088e-02,2.336882e-02
1,Трапеции,4,0.500000,0.266458,5.876057e-03,5.740650e-03
2,Трапеции,8,0.250000,0.270769,1.437009e-03,1.429623e-03
3,Трапеции,16,0.125000,0.271841,3.575180e-04,3.570690e-04
4,Трапеции,32,0.062500,0.272109,8.927422e-05,8.924634e-05
5,Трапеции,64,0.031250,0.272176,2.231202e-05,2.231028e-05
6,Трапеции,128,0.015625,0.272193,5.577598e-06,5.577489e-06
7,Трапеции,256,0.007812,0.272197,1.394374e-06,1.394367e-06
8,Трапеции,512,0.003906,0.272198,3.485919e-07,3.485915e-07
9,Трапеции,1024,0.001953,0.272198,8.714787e-08,8.714785e-08


Вывод: Метод Симпсона сходится намного быстрее метода трапеции.

### Задание 2

Вычислить приближенное значение интеграла из задания 1, используя квадратурную формулу наивысшей алгебраической степени точности (НАСТ) с $ k $ узлами.\
\
В содержание отчета должна быть включена следующая информация:
- КФ НАСТ с $ k $ узлами.
- Приближенное значение интеграла, вычисленное с помощью КФ НАСТ. Сравнение с точным значением $ I $.
- Выводы.
- Листинг программы с комментариями.

In [88]:
def gauss_legendre_integration(a, b, k):
    # Получаем узлы и веса квадратурной формулы Гаусса-Лежандра
    x, w = np.polynomial.legendre.leggauss(k)
    # Пересчитываем интервалы
    x_mapped = (b - a) / 2 * x + (b + a) / 2
    # Вычисляем значение интеграла
    integral = ((b - a) / 2) * np.sum(w * f(x_mapped))
    return integral

print("Значение интеграла НАСТ: ", gauss_legendre_integration(a, b, 5))
print("Погрешность: ", abs(I - gauss_legendre_integration(a, b, 5)))

Значение интеграла НАСТ:  0.27219802575118046
Погрешность:  2.3553676981036276e-07


Вывод: Погрешность КФ НАСТ меньше погрешностей методов выше